## 食行動音声のJSON形式でのファイルを作成するプログラム

In [5]:
eat_wav = "./../dataset/old_wav_aug/conbined"
eat_txt = "./../dataset/old_text_aug"
output_json_path = "./../json/eat_not_aug.json"

In [8]:
import os
import json

def generate_eat_json(eat_wav, eat_txt, output_json_path):
    """
    テキストフォルダ(eat_txt)と音声フォルダ(eat_wav)の内容を対応付けて
    JSONファイル(output_json_path)を作成する関数。
    存在しないフォルダがあれば自動作成。
    """
    
    # 出力先のディレクトリが存在しない場合は作成
    output_dir = os.path.dirname(output_json_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"[Info] 出力先フォルダを作成しました: {output_dir}")
    
    # JSON出力用のリスト
    output_data = []
    
    # テキストファイルの一覧を取得
    txt_files = sorted([f for f in os.listdir(eat_txt) if f.endswith('.txt')])

    for txt_file in txt_files:
        base_name = txt_file.replace('.txt', '')
        parts = base_name.split('_')
        
        # 被験者名と食材名の間に "conbined" を挿入（typo注意: "combined" ではない？）
        parts.insert(2, "conbined")
        
        wav_file_name = '_'.join(parts) + '.wav'
        wav_path = os.path.normpath(os.path.join(eat_wav, wav_file_name))
        if wav_path.startswith(os.pardir + os.sep):
                wav_path = wav_path[len(os.pardir + os.sep):]
        
        txt_path = os.path.join(eat_txt, txt_file)
        
        timestamps_dict = {
            "chewing": [],
            "swallowing": [],
            "noise": [],
            "speech": []
        }
        
        weak_labels = []
        
        with open(txt_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                parts_line = line.split()
                if len(parts_line) < 3:
                    continue

                start_sec = float(parts_line[0])
                end_sec = float(parts_line[1])
                label = parts_line[2]
                
                if label == 'ch':
                    timestamps_dict["chewing"].append([start_sec, end_sec])
                    weak_labels.append('#')
                elif label == 'sw':
                    timestamps_dict["swallowing"].append([start_sec, end_sec])
                    weak_labels.append('$')
                elif label == 'no':
                    timestamps_dict["noise"].append([start_sec, end_sec])
                    weak_labels.append('*')
                else:
                    print(f"[Info] 想定外のラベルがありました: {label}")
                    continue
        
        item = {
            "path": wav_path,
            "timestamps": timestamps_dict,
            "text": "".join(weak_labels)
        }
        
        output_data.append(item)
    
    with open(output_json_path, 'w', encoding='utf-8') as out_f:
        json.dump(output_data, out_f, ensure_ascii=False, indent=2)
    
    print(f"JSONファイルを出力しました: {output_json_path}")

generate_eat_json(eat_wav, eat_txt, output_json_path)


JSONファイルを出力しました: ./../json/eat_not_aug.json
